<a href="https://colab.research.google.com/github/aaayup/anomaly_detection/blob/main/code/ARIMA(normal_all_predict).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data 불러오기

In [1]:
#importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA

In [2]:
df=pd.read_csv('../../simulation/re/데이터_젊음_중년_장년/uniform_격렬/uniform10_중년_격렬.csv')
df = df.copy()
df.head()

,heart_rate1,event1,heart_rate10,event10,heart_rate100,event100,heart_rate1000,event1000,heart_rate101,event101,...,event995,heart_rate996,event996,heart_rate997,event997,heart_rate998,event998,heart_rate999,event999,update_time
0,84,0,138,0,80,0,124,0,97,0,...,0,77,0,149,0,92,0,99,0,01JAN2020:00:00:00
1,140,0,100,0,117,0,113,0,122,0,...,0,152,0,102,0,95,0,98,0,01JAN2020:00:10:00
2,138,0,163,0,77,0,150,0,100,0,...,0,211,1,76,0,144,0,151,0,01JAN2020:00:20:00
3,207,1,82,0,151,0,91,0,80,0,...,0,100,0,102,0,137,0,99,0,01JAN2020:00:30:00
4,153,0,123,0,107,0,38,1,104,0,...,0,186,1,154,0,71,0,126,0,01JAN2020:00:40:00


#반복문

In [3]:
df_copy = df.copy()

# 데이터를 df_train과 df_test로 분할
df_train = df_copy[:1008]
df_test = df_copy[1008:]

print(df_copy.head())

print(df_train.shape)
print(df_test.shape)


   heart_rate1  event1  heart_rate10  event10  heart_rate100  event100  \
0           84       0           138        0             80         0   
1          140       0           100        0            117         0   
2          138       0           163        0             77         0   
3          207       1            82        0            151         0   
4          153       0           123        0            107         0   

   heart_rate1000  event1000  heart_rate101  event101  ...  event995  \
0             124          0             97         0  ...         0   
1             113          0            122         0  ...         0   
2             150          0            100         0  ...         0   
3              91          0             80         0  ...         0   
4              38          1            104         0  ...         0   

   heart_rate996  event996  heart_rate997  event997  heart_rate998  event998  \
0             77         0            149 

In [4]:
for i in range(1, 1001):
    # ARIMA 모델 학습
    model = ARIMA(df_train[f'heart_rate{i}'], order=(2, 0, 1))
    result = model.fit()

    start_idx = 1008  # 테스트 데이터셋의 시작 인덱스
    end_idx = 1008 + len(df_test)  # 테스트 데이터셋의 끝 인덱스

    forecast = result.get_prediction(start=start_idx, end=end_idx, dynamic=False)

    # 신뢰구간과 예측값 할당
    forecast_ci = forecast.conf_int()

  # 데이터프레임에 신뢰구간 추가
    df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
    df_test[f'upper_{i}'] = forecast_ci.iloc[:, 1]

    # 'pred_y{i}' 열이 이미 존재하는 경우 삭제하고 다시 추가 (insert 메서드 사용)
    if f'pred_y{i}' in df_test.columns:
        df_test.drop(columns=[f'pred_y{i}'], inplace=True)

    # 'pred_y{i}' 열을 'event{i}' 뒤에 생성
    df_test.insert(df_test.columns.get_loc(f'event{i}') + 1, f'pred_y{i}', np.where((df_test[f'heart_rate{i}'] < df_test[f'lower_{i}']) | (df_test[f'heart_rate{i}'] > df_test[f'upper_{i}']), 1, 0))

    # 'pred{i}', 'lower_{i}', 'upper_{i} 삭제
    if f'pred{i}' in df_test.columns:
        df_test.drop(columns=[f'pred{i}'], inplace = True)
    if f'lower_{i}' in df_test.columns:
        df_test.drop(columns=[f'lower_{i}'], inplace = True)
    if f'upper_{i}' in df_test.columns:
        df_test.drop(columns=[f'upper_{i}'], inplace = True)

c:\Users\jun99\OneDrive - 계명대학교\바탕 화면\Project\TP\분석 연습\ARIMA\arimafd\arima\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\jun99\OneDrive - 계명대학교\바탕 화면\Project\TP\분석 연습\ARIMA\arimafd\arima\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\jun99\AppData\Local\Temp\ipykernel_4264\1820725872.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'lower_{i}'] = forecast_ci.iloc[:, 0]
C:\Users\ju

In [5]:
df_test

,heart_rate1,event1,pred_y1,heart_rate10,event10,pred_y10,heart_rate100,event100,pred_y100,heart_rate1000,...,heart_rate997,event997,pred_y997,heart_rate998,event998,pred_y998,heart_rate999,event999,pred_y999,update_time
1008,82,0,0,138,0,0,102,0,0,112,...,80,0,0,214,1,1,85,0,0,08JAN2020:00:00:00
1009,80,0,0,159,0,0,84,0,0,91,...,86,0,0,88,0,0,77,0,0,08JAN2020:00:10:00
1010,209,1,1,79,0,0,90,0,0,30,...,194,1,1,180,1,0,162,0,0,08JAN2020:00:20:00
1011,128,0,0,146,0,0,128,0,0,152,...,82,0,0,102,0,0,160,0,0,08JAN2020:00:30:00
1012,163,0,0,103,0,0,158,0,0,211,...,135,0,0,126,0,0,133,0,0,08JAN2020:00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,184,1,0,148,0,0,121,0,0,141,...,77,0,0,83,0,0,83,0,0,10JAN2020:08:10:00
1346,84,0,0,99,0,0,180,1,0,136,...,72,0,0,104,0,0,108,0,0,10JAN2020:08:20:00
1347,159,0,0,139,0,0,94,0,0,137,...,82,0,0,160,0,0,120,0,0,10JAN2020:08:30:00
1348,109,0,0,143,0,0,89,0,0,73,...,86,0,0,142,0,0,141,0,0,10JAN2020:08:40:00


In [6]:
from performance import performance

In [7]:
result = pd.DataFrame()

for i in range(1,1001):

    TP = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==1)])
    TN = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==0)])
    FN = len(df_test[(df_test[f'event{i}']==1)&(df_test[f'pred_y{i}']==0)])
    FP = len(df_test[(df_test[f'event{i}']==0)&(df_test[f'pred_y{i}']==1)])


    result_i = performance(TP=TP, TN=TN, FP=FP, FN=FN, BETA=1)
    
    result = pd.concat([result, result_i])


result.head(2)

,Sensitivity,Specificity,Accuracy,PPV,NPV,F_1 score
0,71.052632,100.0,96.783626,100.0,96.507937,83.076923
0,73.684211,100.0,97.076023,100.0,96.815287,84.848485


In [8]:
균등_격렬_중년_10 = pd.DataFrame(index=['mean'])

균등_격렬_중년_10['mean_sens'] = result['Sensitivity'].mean()
균등_격렬_중년_10['std_sens'] = result['Sensitivity'].std()
균등_격렬_중년_10['mean_spec'] = result['Specificity'].mean()
균등_격렬_중년_10['std_spec'] = result['Specificity'].std()
균등_격렬_중년_10['mean_accu'] = result['Accuracy'].mean()
균등_격렬_중년_10['std_accu'] = result['Accuracy'].std()
균등_격렬_중년_10['mean_ppv'] = result['PPV'].mean()
균등_격렬_중년_10['std_ppv'] = result['PPV'].std()
균등_격렬_중년_10['mean_npv'] = result['NPV'].mean()
균등_격렬_중년_10['std_npv'] = result['NPV'].std()
균등_격렬_중년_10['mean_F1_score'] = result['F_1 score'].mean()
균등_격렬_중년_10['std_F1_score'] = result['F_1 score'].std()

균등_격렬_중년_10

,mean_sens,std_sens,mean_spec,std_spec,mean_accu,std_accu,mean_ppv,std_ppv,mean_npv,std_npv,mean_F1_score,std_F1_score
mean,71.109133,8.160081,100.0,0.0,97.133333,0.812598,100.0,0.0,96.915738,0.860687,82.844617,5.699577


In [9]:
균등_격렬_중년_10.to_csv('../../../분석 연습/result/균등_격렬_중년_10.csv', encoding='cp949')